<a href="https://colab.research.google.com/github/suyogpradhan0/Spatial-Data-Management-with-Google-Earth-Engine/blob/main/Watershed_with_DEM%2C_split_panel_map_for_visualizing_land_cover_change_(2001_2019)_%26_OpenStreetMap_data_for_a_city_boundary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 5

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-414/blob/master/book/labs/lab_05.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).

In [ ]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=GxsQwngERd6QlMqXsqwkKZZbfjbkqVxXPrp8Xchqe8U&tc=nGpbDV0BKARdOw6RUCvgO3ctWgcQqNt1deGuVjydi4k&cc=8-U4cAXm8ogA3H3ahhThbONUb6E-Quk_snLPrLZQpuc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AeaYSHBMJgnH8z8mr0kL6A-dFEOhtYvWCV1KIJ8BcwLuPiG3rb4j82_xIdU

Successfully saved authorization token.


## Question 1

Visualize the [USGS Watershed Boundary Dataset](https://developers.google.com/earth-engine/datasets/catalog/USGS_WBD_2017_HUC04) with outline color only, no fill color.

In [ ]:
# Load the USGS Watershed Boundary Dataset
watersheds = ee.FeatureCollection("USGS/WBD/2017/HUC06")

# Create a map centered on the US with a specified zoom level
Map = geemap.Map(center=[40, -100], zoom=4)

# Add the Watershed Boundary Dataset to the map with the outline color
Map.addLayer(watersheds, {'color': '0000EE00', 'width': 2}, 'Watersheds')

#Add text to the map
text = "Created by Suyog Pradhan"
Map.add_text(text, fontsize=20, position='bottomright')

# Display the map
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

![](https://i.imgur.com/PLlNFq3.png)

## Question 2

Filter the USGS Watershed Boundary dataset and select the watershed that intersects the county of your choice.

In [ ]:
# Filter the watersheds that intersect Knox County, Tennessee
knox_county = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.And(
    ee.Filter.eq('STATEFP', '47'),  # Tennessee state code
    ee.Filter.eq('COUNTYFP', '093')  # Knox County code
))
knox_watersheds = watersheds.filterBounds(knox_county)

# Select two watersheds from Knox County, Tennessee
watershed1 = knox_watersheds.filter(ee.Filter.eq('huc6', '060101')).geometry() #French Broad-Holston Watershed
watershed2 = knox_watersheds.filter(ee.Filter.eq('huc6', '060102')).geometry() #Upper Tennessee Watershed

# Merge the selected watersheds
merged_watershed = watershed1.union(watershed2)

# Display the map
Map = geemap.Map()
Map.centerObject(merged_watershed, 8)
Map.addLayer(knox_county, {}, 'Knox County, TN')
Map.addLayer(merged_watershed, {'color': '7B68EE'}, 'Selected Watersheds')
Map.addLayerControl()
Map.add_text(text, fontsize=20, position='bottomright')
Map

Map(center=[36.068788867875135, -83.12321502172557], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/F2QfqZu.png)

## Question 3

Clip the [USGS 3DEP 10m DEM](https://developers.google.com/earth-engine/datasets/catalog/USGS_3DEP_10m) with the watershed that intersects the county of your choice. Display the DEM with a proper color palette and color bar.

In [ ]:
# Load the USGS 3DEP 10m DEM
dem = ee.Image('USGS/SRTMGL1_003')

# Clip the DEM to the selected watershed
dem_clipped = dem.clip(merged_watershed)

# Create a map centered on the selected watershed
Map = geemap.Map()
Map.centerObject(merged_watershed, 8)
Map.addLayer(merged_watershed, {'color': 'FF000000'}, 'Selected Watersheds')
Map.add_text(text, fontsize=20, position='bottomright')

# Add the DEM to the map with a proper color palette
Map.addLayer(dem_clipped, {'min': 0, 'max': 1500, 'palette': 'terrain'}, 'DEM')

# Add a color bar to the map
Map.add_colorbar({'min': 0, 'max': 1800, 'palette': 'terrain'}, label='Elevation (m)')

# Display the map
Map.addLayerControl()
Map

Map(center=[36.068788867875135, -83.12321502172557], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/okR39pf.png)

## Question 4

Use the [USGS National Land Cover Database](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD) and [US Census States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States) to create a split-panel map for visualizing land cover change (2001-2019) for a US state of your choice. Make sure you add the NLCD legend to the map.

In [ ]:
# Load US Census States
states = ee.FeatureCollection('TIGER/2018/States')

# Filter California
california = states.filter(ee.Filter.eq('STUSPS', 'CA'))

# Load NLCD data for 2001 and 2019
nlcd_2001 = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2001').select('landcover')
nlcd_2019 = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019').select('landcover')

# Mask out no-data pixels
left_layer = geemap.ee_tile_layer(nlcd_2001.clip(california), {}, 'NLCD 2001')
right_layer = geemap.ee_tile_layer(nlcd_2019.clip(california), {}, 'NLCD 2019')

# Display NLCD 2001 on the left panel and NLCD 2019 on the right panel
Map = geemap.Map()
Map.split_map(left_layer, right_layer, add_close_button=True, left_label='2001', right_label='2019')

# Center the map on California
Map.centerObject(california, 6)

# Add NLCD legend to the map
legend_dict = {
    '11 Open Water': '466b9f',
    '12 Perennial Ice/Snow': 'd1def8',
    '21 Developed, Open Space': 'dec5c5',
    '22 Developed, Low Intensity': 'd99282',
    '23 Developed, Medium Intensity': 'eb0000',
    '24 Developed High Intensity': 'ab0000',
    '31 Barren Land (Rock/Sand/Clay)': 'b3ac9f',
    '41 Deciduous Forest': '68ab5f',
    '42 Evergreen Forest': '1c5f2c',
    '43 Mixed Forest': 'b5c58f',
    '51 Dwarf Scrub': 'af963c',
    '52 Shrub/Scrub': 'ccb879',
    '71 Grassland/Herbaceous': 'dfdfc2',
    '72 Sedge/Herbaceous': 'd1d182',
    '73 Lichens': 'a3cc51',
    '74 Moss': '82ba9e',
    '81 Pasture/Hay': 'dcd939',
    '82 Cultivated Crops': 'ab6c28',
    '90 Woody Wetlands': 'b8d9eb',
    '95 Emergent Herbaceous Wetlands': '6c9fb8',
}

Map.add_legend(title="NLCD Land Cover Classification", legend_dict=legend_dict)

# Display the map
Map.add_text(text, fontsize=20, position='bottomright')
Map

Map(center=[37.152434104671315, -119.52655155150876], controls=(ZoomControl(options=['position', 'zoom_in_text…

![](https://i.imgur.com/Au7Q5Ln.png)

## Questions 5

Download OpenStreetMap data for a city of your choice and visualize the city boundary and restaurants in the city.

In [ ]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
import geemap.osm as osm

In [ ]:
# Create a Map object with Google Map layer disabled
Map = geemap.Map(add_google_map=False)

# Retrieve OpenStreetMap (OSM) features for "Knoxville, Tennessee"
Knoxville = geemap.osm_to_ee("Knoxville, Tennessee")

# Add the OSM features to the map as a layer named "Knoxville"
Map.addLayer(Knoxville, {}, "Knoxville")

# Use the osm module to get OpenStreetMap (OSM) data for restaurants in Knoxville, within a specified distance
gdf = osm.osm_gdf_from_address(
    address="Knoxville, Tennessee", tags={"amenity": "restaurant"}, dist=15000
)

# Add the GeoDataFrame (gdf) of restaurants to the map as a layer named "Knoxville's restaurants"
Map.add_gdf(gdf, layer_name="Knoxville's restaurants")

# Center the map view on the OpenStreetMap features for Knoxville
Map.centerObject(Knoxville, 11)

# Display the map
Map.add_text(text, fontsize=20, position='bottomright')
Map

Map(center=[35.96852075854313, -83.95017073202958], controls=(WidgetControl(options=['position', 'transparent_…

![](https://i.imgur.com/AUlO1CV.png)